In [61]:
import os 
from typing import Annotated
from openai import AsyncOpenAI
from dotenv import load_dotenv
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

In [62]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""
    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Kathmandu, Nepal",
            "Pokhara, Nepal",
            "Bhaktapur, Nepal",
            "Lumbini, Nepal",
            "Patan, Nepal",
            "Chitwan, Nepal",
            "Birtamode, Nepal",
            "Ilam, Nepal",
            "Bandipur, Nepal",
        ]
        # Track last destination to avoid repeats
        self.last_destination = None
    #This created a function that AI agent could use
    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)
        # Select a random destination
        destination = random.choice(available_destinations)
        # Update the last destination
        self.last_destination = destination
        return destination

In [63]:
class FoodsPlugin:
    """Suggests famous foods based on destination."""
    @kernel_function(description="Suggests a famous food based on destination.")
    def get_food_suggestion(self) -> Annotated[str, "Returns a food suggestion."]:
        foods = [
            "Tea in Ilam",
            "Jujudhau in Bhaktapur",
            "Momo in Kathmandu",
            "Rice in Pokhara",
            "Sweet in Lumbini",
        ]
        return random.choice(foods)

In [64]:
class PackingPlugin:
    """Suggests travel packing essentials."""
    @kernel_function(description="Suggests essential travel packing items.")
    def get_packing_list(self) -> Annotated[str, "Returns a list of essential packing items."]:
        return "Don't forget your passport, sunglasses, comfy shoes, power bank, and travel pillow!"

In [65]:
class ItineraryPlugin:
    """Creates a simple travel itinerary for a day."""
    @kernel_function(description="Creates a simple travel itinerary for a day.")
    def create_day_itinerary(self) -> Annotated[str, "Returns a simple plan for a day."]:
        return (
            "Morning: Start Breakfast.\n"
            "Afternoon: Lunch at a famous cafe.\n"
            "Evening: Walk by the river and dinner at a restaurant."
        )

In [66]:
class MapsPlugin:
    """Generates Google Maps links for destinations."""
    @kernel_function(description="Generates a Google Maps link for a destination.")
    def get_google_maps_link(self) -> Annotated[str, "Returns a Google Maps search link."]:
        destinations = [
            "Paris",
            "Tokyo",
            "New York",
            "Barcelona",
            "Sydney",
        ]
        place = random.choice(destinations)
        link = f"https://www.google.com/maps/search/{place.replace(' ', '+')}"
        return f"Here's the map for {place}: {link}"

In [67]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)
# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [68]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin(), FoodsPlugin(), PackingPlugin(), ItineraryPlugin(),MapsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI agent that helps plan vacations including destination, food, packing, and maps",
)

In [69]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None
    user_inputs = [
        "Plan me a weekend trip. Include Adventures, road trips, and fun",
    ]
    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()
    # Clean up the thread
    await thread.delete() if thread else None
await main()

# User: Plan me a weekend trip. Include Adventures, road trips, and fun

# TravelAgent: ### Weekend Trip to Lumbini, Nepal

#### Destination Overview
Lumbini is a UNESCO World Heritage site and the birthplace of Lord Buddha, making it a significant pilgrimage and cultural site that offers a mix of spiritual adventure and natural beauty.

#### Food Suggestion
- **Must-Try Dish:** Rice in Pokhara

#### Essential Packing List
- Passport
- Sunglasses
- Comfy shoes
- Power bank
- Travel pillow

#### Itinerary
**Day 1: Arrival and Exploration**
- **Morning:** Start with a hearty breakfast at your hotel.
- **Afternoon:** Enjoy lunch at a famous local café.
- **Evening:** Stroll by the river to take in the serene views and have dinner at a local restaurant.

#### Map
- [View Lumbini on Google Maps](https://www.google.com/maps/search/Tokyo)

### Adventures & Fun Activities
- **Explore the Lumbini Garden:** Walk through the sacred garden with various monuments and the sacred pond.
- **Visit Maya